In [ ]:
import pulp
import numpy as np
import itertools as it

In [ ]:
games = np.load("games.npz")
for game_name in sorted(games.keys()):
    G = games[game_name]
    P = G.shape[-1]
    A = G.shape[:-1]
    print(game_name + ":", P, "players,", A, "actions")
pk3 = games["pk_3_actions"]
pk3[:,:,0]

bach_or_stravinsky: 2 players, (2, 2) actions
chicken: 2 players, (2, 2) actions
five_by_five: 2 players, (5, 5) actions
four_players: 4 players, (2, 2, 2, 2) actions
hawk_dove: 2 players, (2, 2) actions
matching_pennies: 2 players, (2, 2) actions
penalty_kick: 2 players, (2, 2) actions
pk_3_actions: 2 players, (3, 3) actions
prisoners_dilemma: 2 players, (2, 2) actions
robot_escape: 2 players, (2, 2) actions
rock_paper_scissors: 2 players, (3, 3) actions
rps_both_hate_ties: 2 players, (3, 3) actions
rps_p1_likes_rock: 2 players, (3, 3) actions
rps_p2_dislikes_ties: 2 players, (3, 3) actions
two_by_three_by_four: 3 players, (2, 3, 4) actions


array([[0.3, 0.9, 0.8],
       [0.6, 0.2, 0.6],
       [0.4, 0.4, 0.1]])

We can create an LP object for this game using PULP.

In [ ]:
pk_lp = pulp.LpProblem("Penalty_Kick", pulp.LpMaximize) # create LP object
p_kl = pulp.LpVariable("Pr_KL", 0, 1, pulp.LpContinuous) # lower bound = 0, upper bound = 1
p_kr = pulp.LpVariable("Pr_KR", 0, 1, pulp.LpContinuous) # lower bound = 0, upper bound = 1
p_jl = pulp.LpVariable("Pr_JL", 0, 1, pulp.LpContinuous) # lower bound = 0, upper bound = 1
p_jr = pulp.LpVariable("Pr_JR", 0, 1, pulp.LpContinuous) # lower bound = 0, upper bound = 1

pk_lp += 0 # no objective necessary
pk_lp += p_kl + p_kr == 1 # player 1's probabilities sum to 1
pk_lp += p_jl + p_jr == 1 # player 2's probabilities sum to 1
pk_lp += .7*p_kl + .4*p_kr == .1*p_kl + .8*p_kr # player 2 is indifferent
pk_lp += .3*p_jl + .9*p_jr == .6*p_jl + .2*p_jr # player 1 is indifferent

In [ ]:
print(pk_lp)

Penalty_Kick:
MAXIMIZE
0
SUBJECT TO
_C1: Pr_KL + Pr_KR = 1

_C2: Pr_JL + Pr_JR = 1

_C3: 0.6 Pr_KL - 0.4 Pr_KR = 0

_C4: - 0.3 Pr_JL + 0.7 Pr_JR = 0

VARIABLES
Pr_JL <= 1 Continuous
Pr_JR <= 1 Continuous
Pr_KL <= 1 Continuous
Pr_KR <= 1 Continuous



And after we call the LP-solver, we can check the status, where +1 indicates that it found a solution and -1 indicates that it didn't.

In [ ]:
pk_lp.solve(solver=pulp.PULP_CBC_CMD(msg=0)) # solve quietly
if pk_lp.status == 1:
    profile = np.zeros(4)
    profile[0] = p_kl.varValue
    profile[1] = p_kr.varValue
    profile[2] = p_jl.varValue
    profile[3] = p_jr.varValue
    print("found equilibrium:", profile)
else:
    print("couldn't find an equilibrium")

found equilibrium: [0.4 0.6 0.7 0.3]


In [ ]:
def build_profile(action_prob_LP_vars):
    profile = np.zeros(len(action_prob_LP_vars))
    for a,prob_var in enumerate(action_prob_LP_vars):
        profile[a] = prob_var.varValue
    return profile

In [ ]:
def nonempty_subsets(S):
    return it.chain.from_iterable(it.combinations(S, a+1) for a in range(len(S)))

Generalize the approach demonstrated above to identify and return a Nash equilibrium in any 2-player game by searching over supports. A good rule of thumb is to start with small, balanced support sets.

In [19]:
def two_player_Nash_LP(game):
    num_actions_p1 = game.shape[0]
    num_actions_p2 = game.shape[1]
    util_p1 = []
    util_p2 = []
    
    
    p1_prob_vars = [pulp.LpVariable("P1_A" + str(a), 0, 1, pulp.LpContinuous) for a in range(num_actions_p1)]
    p2_prob_vars = [pulp.LpVariable("P2_A" + str(a), 0, 1, pulp.LpContinuous) for a in range(num_actions_p2)]
    all_prob_vars = p1_prob_vars + p2_prob_vars
    
    

    for a in range(num_actions_p1):
        util_p1.append(pulp.lpSum(pay*var for pay,var in zip(game[a,:,0], p2_prob_vars)))
    for a in range(num_actions_p2):
        util_p2.append(pulp.lpSum(pay*var for pay,var in zip(game[:,a,1], p1_prob_vars)))   
        
    for p1_support, p2_support in it.product(nonempty_subsets(range(num_actions_p1)), nonempty_subsets(range(num_actions_p2))):
        twoP_game = pulp.LpProblem("G", pulp.LpMaximize)
        twoP_game += 0
        twoP_game += pulp.lpSum(p1_prob_vars) == 1 
        twoP_game += pulp.lpSum(p2_prob_vars) == 1 
        first_action_p1 = p1_support[0]
        first_action_p2 = p2_support[0]
        for p1_a in range(num_actions_p1):
            if p1_a in p1_support and p1_a != first_action_p1:
                twoP_game += util_p1[first_action_p1] == util_p1[p1_a]
            elif p1_a != first_action_p1:
                twoP_game += p1_prob_vars[p1_a] == 0
                twoP_game += util_p1[first_action_p1] >= util_p1[p1_a]
        for p2_a in range(num_actions_p2):
            if p2_a in p2_support and p2_a != first_action_p2:
                twoP_game += util_p2[first_action_p2] == util_p2[p2_a]
            elif p2_a != first_action_p2:
                twoP_game += p2_prob_vars[p2_a] == 0
                twoP_game += util_p2[first_action_p2] >= util_p2[p2_a]
        twoP_game.solve(solver=pulp.PULP_CBC_CMD(msg=0))
        if twoP_game.status == 1:
            profile = build_profile(all_prob_vars)
            return profile
            
    

In [20]:
two_player_Nash_LP(pk3)

array([0.4, 0.6, 0. , 0.7, 0.3, 0. ])